# 1. Importar Bibliotecas

In [19]:
#Importar bibliotecas
import pandas as pd
import numpy as np

# 2. Carregar Dados e Tratamento Inicial

In [20]:
#Carregar dados
base_usuarios = pd.read_csv('base_usuarios_final.csv')
base_transacoes = pd.read_csv('base_transacoes_final.csv')

## 2.1. Base Usuários

In [21]:
#Checar nulos
100 * base_usuarios.isnull().sum() / len(base_usuarios)

user_id             0.000000
createdAt           0.000000
state_anom          0.000000
is_state_capital    0.000000
age_range           0.000000
gender              2.687274
dtype: float64

Dado o tamanho da base e o pequeno percentual de registros com valores nulos, vamos adotar a simples ação de remover os registros em questão.

In [22]:
#Remover registros com campos nulo
base_usuarios.dropna(inplace = True)

In [23]:
#Verificar tipos dos campos
base_usuarios.dtypes

user_id              int64
createdAt           object
state_anom          object
is_state_capital     int64
age_range           object
gender              object
dtype: object

In [24]:
#Tratar campo de data / remover informações de horário
base_usuarios['createdAt'] = [np.datetime64(data[:10]) for data in base_usuarios['createdAt']]

In [25]:
#Renomear colunas
novas_colunas = {
    'user_id': 'id_usuario', 
    'createdAt': 'data_criacao_conta',
    'state_anom': 'estado_anonimo',
    'is_state_capital': 'flag_capital_do_estado',
    'age_range': 'faixa_idade',
    'gender': 'genero'}

base_usuarios.rename(columns = novas_colunas, inplace = True)

## 2.2. Base Transações

In [26]:
#Checar nulos
100 * base_transacoes.isnull().sum() / len(base_usuarios)

createdat               0.000000
ordem_transacao         0.000000
type                    0.000000
transactiontype_anom    0.000000
amount_anom             0.006143
user_id                 0.000000
dtype: float64

In [27]:
len(base_transacoes)

826294

In [28]:
base_transacoes.isnull().sum()

createdat                0
ordem_transacao          0
type                     0
transactiontype_anom     0
amount_anom             55
user_id                  0
dtype: int64

In [29]:
#Remover registros com campos nulo
base_transacoes.dropna(inplace = True)

In [30]:
#Verificar tipos dos campos
base_transacoes.dtypes

createdat                object
ordem_transacao           int64
type                     object
transactiontype_anom     object
amount_anom             float64
user_id                   int64
dtype: object

In [31]:
#Tratar campo de data
base_transacoes['createdat'] = [np.datetime64(data) for data in base_transacoes['createdat']]

In [32]:
#Renomear colunas
novas_colunas = {
    'createdat': 'data_transacao',
    'type': 'grupo_transacao',
    'transactiontype_anom': 'tipo_transacao_anonimo',
    'amount_anom': 'valor',
    'user_id': 'id_usuario'}

base_transacoes.rename(columns = novas_colunas, inplace = True)

# 3. Tratamento dos Dados

## 3.1. Consolidação dos Dados de Transações

A base de transações contém todas as 2 primeiras transações dos usuários. O primeiro passo na nossa análise é consolidar os dois registros em um só, de forma a ter um entendimento melhor do comportamento inicial do usuário.

In [33]:
#Criar campo de ordenação das transações
base_transacoes['ordem_transacao'] = base_transacoes.groupby(by = 'id_usuario').cumcount() + 1

In [34]:
#Consolidar dados
base_transacoes = base_transacoes[base_transacoes['ordem_transacao'] == 1].join(
    on = 'id_usuario',
    other = base_transacoes[base_transacoes['ordem_transacao'] == 2].set_index('id_usuario'),
    rsuffix = '_2')

In [35]:
#Renomear colunas relativas à primeira transação
novas_colunas = {
    'data_transacao': 'data_transacao_1',
    'grupo_transacao': 'grupo_transacao_1',
    'tipo_transacao_anonimo': 'tipo_transacao_anonimo_1',
    'valor': 'valor_1'}

base_transacoes.rename(columns = novas_colunas, inplace = True)

In [36]:
len(base_transacoes)

432025

In [37]:
#Remover campos ordem_transacao
base_transacoes.drop(columns = ['ordem_transacao', 'ordem_transacao_2'], inplace = True)

In [38]:
#Criar campo calculado dias_entre_transacoes
for col in ['data_transacao_1', 'data_transacao_2']:
    base_transacoes[col] = [np.datetime64(str(data)[:10]) for data in base_transacoes[col]]
    
base_transacoes['dias_entre_transacoes'] = [(tx_2 - tx_1) / np.timedelta64(1, 'D') for tx_2, tx_1 in 
                                            zip(base_transacoes['data_transacao_2'], base_transacoes['data_transacao_1'])]

In [39]:
#Criar compo saldo_restante
saldo_1 = [valor if grupo == 'cashin' else - valor for valor, grupo 
           in zip(base_transacoes['valor_1'], base_transacoes['grupo_transacao_1'])]
saldo_2 = [valor if grupo == 'cashin' else - valor for valor, grupo 
           in zip(base_transacoes['valor_2'], base_transacoes['grupo_transacao_2'])]

base_transacoes['saldo_restante'] = [val_1 + val_2 + 5 for val_1, val_2 in zip(saldo_1, saldo_2)]

In [40]:
print('Percentual de registros com saldo restante negativo: {:.2f}%'.format(
    100 * len(base_transacoes[base_transacoes['saldo_restante'] < 0]) / len(base_transacoes)))

Percentual de registros com saldo restante negativo: 20.44%


Na criação do campo saldo_restante, somamos R$ 5 ao saldo final de todos os usuários por conta de um bônus que é aplicado aos usuários por conta de sua primeira transação.

Adicionalmente, cerca de 20% dos registros apresentam o saldo em conta restante negativo, mesmo considerando o acréscimo do bônus. Isso decorre do fato que, no processo de anonimização dos dados, foi incluído certo nível de ruído nos valores dos campos. Com isso, não faz sentido realizar essa operação com os dois atributos, dado que os efeitos do ruído podem se cancelar ou somar. Ficamos então com a decisão de remover o campo da nossa análise.

In [41]:
#Remover campo saldo_restante
base_transacoes.drop(columns = 'saldo_restante', inplace = True)

## 3.2. Cruzar Dados de Usuários e Transações

In [42]:
#Cruzar dados das bases
base_atividade = base_transacoes.join(on = 'id_usuario', other = base_usuarios.set_index('id_usuario')) 

In [43]:
#Criar campo calculado dias_ate_primeira_transacao
base_atividade['dias_ate_primeira_transacao'] = [(tx - conta) / np.timedelta64(1, 'D') for tx, conta in 
                                                 zip(base_atividade['data_transacao_1'], base_atividade['data_criacao_conta'])]

In [44]:
#Reordenar colunas
colunas = list(base_atividade.columns)
colunas.remove('id_usuario')
colunas = ['id_usuario'] + colunas

base_atividade = base_atividade[colunas] 

In [45]:
base_atividade.head()

,id_usuario,data_transacao_1,grupo_transacao_1,tipo_transacao_anonimo_1,valor_1,data_transacao_2,grupo_transacao_2,tipo_transacao_anonimo_2,valor_2,dias_entre_transacoes,data_criacao_conta,estado_anonimo,flag_capital_do_estado,faixa_idade,genero,dias_ate_primeira_transacao
0,1126540,2020-10-04,cashin,cashin_02,9.353113,2020-10-25,cashout,cashout_00,12.959309,21.0,2020-10-03,estado_08,1.0,age_range_02,F,1.0
2,754306,2021-01-28,cashin,cashin_01,521.769527,2021-01-28,cashin,cashin_01,9.681419,0.0,2020-08-04,estado_15,0.0,age_range_05,F,177.0
4,956433,2020-09-10,cashin,cashin_01,21.716756,2020-09-10,cashout,cashout_01,21.227847,0.0,2020-09-10,estado_02,0.0,age_range_01,F,0.0
6,1235453,2020-11-27,cashin,cashin_01,232.586991,2020-11-27,cashout,cashout_02,194.670591,0.0,2020-10-17,estado_02,0.0,age_range_04,F,41.0
8,1108066,2020-10-01,cashin,cashin_01,159.221182,2020-10-01,cashout,cashout_03,178.372101,0.0,2020-10-01,estado_00,0.0,age_range_05,M,0.0


In [25]:
#Consolidar dados com target
base_target = pd.read_csv('base_target_final.csv')
base_target['flag_retido'] = [1 if mes >=4 else 0 for mes in base_target['meses_ativo']]

base_atividade = base_atividade.join(on = 'id_usuario', other = base_target.set_index('user_id')['flag_retido'])

## 3.3. Tratamento Adicional dos Dados

Com o cruzamento das duas fontes de dados, é possível que algum tratamento final seja necessário nos dados. Vamos agora verificar essa possibilidade.

In [46]:
#Verificar existência de registros nulos na base consolidada
nulos = base_atividade.isnull().sum()
100 * nulos / len(base_atividade)

id_usuario                     0.000000
data_transacao_1               0.000000
grupo_transacao_1              0.000000
tipo_transacao_anonimo_1       0.000000
valor_1                        0.000000
data_transacao_2               8.752040
grupo_transacao_2              8.752040
tipo_transacao_anonimo_2       8.752040
valor_2                        8.752040
dias_entre_transacoes          8.752040
data_criacao_conta             2.566518
estado_anonimo                 2.566518
flag_capital_do_estado         2.566518
faixa_idade                    2.566518
genero                         2.566518
dias_ate_primeira_transacao    2.566518
dtype: float64

In [48]:
len(base_atividade)

432025

In [47]:
nulos

id_usuario                         0
data_transacao_1                   0
grupo_transacao_1                  0
tipo_transacao_anonimo_1           0
valor_1                            0
data_transacao_2               37811
grupo_transacao_2              37811
tipo_transacao_anonimo_2       37811
valor_2                        37811
dias_entre_transacoes          37811
data_criacao_conta             11088
estado_anonimo                 11088
flag_capital_do_estado         11088
faixa_idade                    11088
genero                         11088
dias_ate_primeira_transacao    11088
dtype: int64

O que percebemos aqui são uma série de campos com registros nulos. No entanto, olhando com mais atenção, podemos identificar que os campos com valores faltando derivam de duas fontes:

* Dados da segunda transação dos usuários;
* Dados cadastrais.

No primeiro caso, a escolha é óbvia. Ainda que representem quase 10% de todos os registros da base, precisamos remover todos as linhas com valores nulos relacionados à segunda transação. Como queremos medir a probabilidade de retenção dos usuários, não há dúvida de que usuários que não chegaram sequer a fazer 2 operações não podem ser considerados retidos.

No segundo caso, a opção mais simples é também a remoção, especialmente considerando a baixa representatividade no tamanho total da base de dados. Analisando o caso mais à fundo, relembramos que, na coleta dos dados cadastrais dos usuários, removemos todos aqueles clientes em situação irregular. De maneira mais clara, foram removidos todos os usuários bloqueados por suspeita/confirmação de fraude. A inexistência de dados para esses usuários é um indicativo de que estariam dentro dessa classificação. Assim, fica mais embasada a escolha pela deleção dos registros.

In [49]:
#Remover registros com dados nulos
base_atividade.dropna(inplace = True)

In [50]:
len(base_atividade)

384112

In [31]:
#Salvar base de dados tratada
base_atividade.to_csv('base_final.csv', index = False)